In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [6]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [13]:
print(len(X))
print(len(y))

96314
96314


In [7]:
#model = load_model("mymodel.h5")
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X,y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp,Y_temp, test_size=0.5, random_state=42)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=100, verbose=1, validation_data=(X_val, Y_val))

Epoch 1/100
2408/2408 [==============================] - 30s 10ms/step - loss: 6.3264 - accuracy: 0.0691 - val_loss: 5.9846 - val_accuracy: 0.0983
Epoch 2/100
2408/2408 [==============================] - 23s 10ms/step - loss: 5.6121 - accuracy: 0.1134 - val_loss: 5.7858 - val_accuracy: 0.1202
Epoch 3/100
2408/2408 [==============================] - 22s 9ms/step - loss: 5.2307 - accuracy: 0.1390 - val_loss: 5.7287 - val_accuracy: 0.1345
Epoch 4/100
2408/2408 [==============================] - 23s 10ms/step - loss: 4.9072 - accuracy: 0.1561 - val_loss: 5.7637 - val_accuracy: 0.1406
Epoch 5/100
2408/2408 [==============================] - 22s 9ms/step - loss: 4.6038 - accuracy: 0.1740 - val_loss: 5.8165 - val_accuracy: 0.1441
Epoch 6/100
2408/2408 [==============================] - 17s 7ms/step - loss: 4.3118 - accuracy: 0.1951 - val_loss: 5.9305 - val_accuracy: 0.1377
Epoch 7/100
2408/2408 [==============================] - 18s 7ms/step - loss: 4.0326 - accuracy: 0.2156 - val_loss: 6.052

In [10]:
seed_text = "This ensures that the model works with the"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 16ms/step
This ensures that the model works with the soul it is


In [11]:
model.save("bestModel.h5")

In [12]:
model.save("myModel.h5")